# relax計算のtrajectoryをディレクトリに分割する

In [1]:
from glob import glob
from mlptools.io.read import read_from_format
from ase.io.espresso import read_espresso_out, read_espresso_in
import os
import pickle

In [2]:
path2relax = "/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax"
relax_dirs = glob(os.path.join(path2relax, "relax_scf_*"))
print(f"Number of relax directories: {len(relax_dirs)}")

Number of relax directories: 312


In [3]:
print(relax_dirs)

['/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_3171', '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_6631', '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_8552', '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_12066', '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_11245', '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_2608', '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_347', '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_12236', '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_

In [4]:
# 各relaxディレクトリに対して、trajectoryごとにディレクトリを作成しその中に、Atoms(ase_atoms.pkl)とMLPAtoms(atoms.pkl)を保存する
for relax_dir in relax_dirs:
    all_ase_atoms = []
    print(relax_dir)

    with open(os.path.join(relax_dir, "relax.out")) as f:
        lines = [s.strip() for s in f.readlines()]
    is_valid = True
    for l in lines:
        if "convergence NOT achieved" in l:
            is_valid = False
            break
    if not is_valid:
        continue
    
    with open(os.path.join(relax_dir, "relax.out")) as f:
        atom_gen = read_espresso_out(f, index=slice(None))
        while True:
            try:
                ase_atoms = next(atom_gen)
                all_ase_atoms.append(ase_atoms)
            except StopIteration:
                break
    
    print(f"Number of trajectories: {len(all_ase_atoms)}")
    if len(all_ase_atoms) == 0:
        continue

    # 全atomsを保存
    for i, ase_atoms in enumerate(all_ase_atoms):
        # create directory
        path2atoms = os.path.join(relax_dir, "atoms", f"atoms_{i}")
        os.makedirs(path2atoms, exist_ok=True)

        # create mlpatoms
        mlpatoms = read_from_format(
            ase_atoms=ase_atoms,
            format="ase",
        )

        # save atoms
        pickle.dump(ase_atoms, open(os.path.join(path2atoms, "ase_atoms.pkl"), "wb"))
        pickle.dump(mlpatoms, open(os.path.join(path2atoms, "atoms.pkl"), "wb"))

/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_3171


Number of trajectories: 26
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_6631
Number of trajectories: 23
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_8552
Number of trajectories: 24
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_12066
Number of trajectories: 50
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_11245
Number of trajectories: 27
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_2608
Number of trajectories: 39
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_347
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/beta-cristobalite__mp-546794/relax/relax_scf_12236
Number of trajectories: 41
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_ko

In [5]:
# for ase_atoms in all_ase_atoms:
#     mlpatoms = read_from_format(
#         ase_atoms=ase_atoms,
#         format="ase",
#     )
#     break